<a href="https://colab.research.google.com/github/Risskr/Stock-App/blob/main/StocksApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up


In [8]:
#refesh python script
#get_ipython().kernel.do_shutdown(restart=True)

#Check GPUs
##!nvidia-smi

In [9]:
# Install nbimporter if not already installed
# !pip install import-ipynb
# import import_ipynb

#load cudf to use GPUs for analysis
%load_ext cudf.pandas

# import pandas
import pandas as pd

The cudf.pandas extension is already loaded. To reload it, use:
  %reload_ext cudf.pandas


# Get Stock Data

## Download Stock Data

In [10]:
#Download the stock data
##!if [ ! -f "usa_stocks_30m.parquet" ]; then curl https://storage.googleapis.com/rapidsai/colab-data/usa_stocks_30m.parquet -o usa_stocks_30m.parquet; else echo "usa_stocks_30m.parquet found"; fi

In [11]:
#move the stock data to my Drive
##!mv usa_stocks_30m.parquet "/content/drive/MyDrive/Colab Notebooks"

In [12]:
# # Add my drive to the sys.path
# # Replace 'My Drive/Colab Notebooks' with the actual path to the directory
# # within your Google Drive where Correlation_Coefficient.ipynb is located.
# import sys
# sys.path.append('/content/drive/My Drive/Colab Notebooks')

# # Verify that the directory is in sys.path
# print("sys.path after appending:")
# print(sys.path)

# # List the contents of the directory to check if the file exists
# print("\nContents of the directory:")
# !ls "/content/drive/My Drive/Colab Notebooks"

# # Check if the notebook is recognized as a module
# print("\nChecking for Correlation_Coefficient in sys.modules:")
# print('Correlation_Coefficient' in sys.modules)

## Pull data from My Drive

In [13]:
# Define nasdaq_stocks as the stock data
# Specify the full path to the file in your Google Drive
nasdaq_stocks = pd.read_parquet("/content/drive/MyDrive/Colab Notebooks/usa_stocks_30m.parquet")

In [14]:
#Cut down the data first 18M lines because of RAM limits on
#free tier of colab
#df = df.iloc[:18000000]

## Add columns and Fitler last 5 years

In [15]:
#add year, month, and day columns
#df[["year", "week", "day"]] = df.datetime.dt.isocalendar()
nasdaq_stocks["year"] = nasdaq_stocks.datetime.dt.year
nasdaq_stocks["month"] = nasdaq_stocks.datetime.dt.month
nasdaq_stocks["day"] = nasdaq_stocks.datetime.dt.day

In [27]:
# Filter the DataFrame based on the 'year' column
nasdaq_stocks_last_5y = nasdaq_stocks.loc[nasdaq_stocks['year'] >= 2023]

nasdaq_stocks_last_5y.info()

<class 'cudf.core.dataframe.DataFrame'>
Index: 2115063 entries, 75734 to 36919
Data columns (total 10 columns):
 #   Column    Dtype
---  ------    -----
 0   datetime  datetime64[ms]
 1   open      float64
 2   high      float64
 3   low       float64
 4   close     float64
 5   volume    int64
 6   ticker    object
 7   year      int16
 8   month     int16
 9   day       int16
dtypes: datetime64[ms](1), float64(4), int16(3), int64(1), object(1)
memory usage: 139.5+ MB


In [17]:
nasdaq_stocks_last_5y.head()

,datetime,open,high,low,close,volume,ticker,year,month,day
65555,2020-01-02 16:30:00,85.90,86.35,85.20,85.99,163797,A,2020,1,2
65556,2020-01-02 17:00:00,86.01,86.01,85.36,85.54,49377,A,2020,1,2
65557,2020-01-02 17:30:00,85.54,85.54,85.31,85.44,46574,A,2020,1,2
65558,2020-01-02 18:00:00,85.43,85.61,85.39,85.45,47512,A,2020,1,2
65559,2020-01-02 18:30:00,85.47,85.47,85.24,85.43,58244,A,2020,1,2


## Filter for daily close values

In [18]:
#aggregate data by ticker, month, day, closing cost.
nasdaq_stocks_last_5y_aggregated_close = nasdaq_stocks_last_5y.groupby(["ticker", "year", "month", "day"]).agg({"close": "last"})

nasdaq_stocks_last_5y_aggregated_close.head()

close
ticker year month day       
A      2020 1     2    85.91
                  3    84.64
                  6    84.80
                  7    85.09
                  8    85.90

# Pull GOOG and MSFT

In [19]:
# Access the 'ticker' level from the index for filtering
GOOGDailyClose = nasdaq_stocks_last_5y_aggregated_close.loc[nasdaq_stocks_last_5y_aggregated_close.index.get_level_values('ticker') == "GOOG"]
MSFTDailyClose = nasdaq_stocks_last_5y_aggregated_close.loc[nasdaq_stocks_last_5y_aggregated_close.index.get_level_values('ticker') == "MSFT"]


GOOGDailyClose.info()

<class 'cudf.core.dataframe.DataFrame'>
MultiIndex: 1104 entries, ('GOOG', np.int16(2020), np.int16(1), np.int16(2)) to ('GOOG', np.int16(2024), np.int16(3), np.int16(13))
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   close   1104 non-null   float64
dtypes: float64(1)
memory usage: 62.0+ KB


In [20]:
GOOGDailyClose.head()

close
ticker year month day       
GOOG   2020 1     2    68.27
                  3    68.05
                  6    69.70
                  7    69.71
                  8    70.19

# Correlation_Coefficient Function

In [21]:
# Correlation_Coefficient Function
# Using the Pearson correlation coefficient to determine correlation between two stocks

def Correlation_Coefficient(StockA, StockB):
  # Calculate the correlation between the 'close' columns
  StockA = StockA['close'].droplevel('ticker')
  StockB = StockB['close'].droplevel('ticker')
  correlation = StockA.corr(StockB)
  return correlation


correlation_value = Correlation_Coefficient(GOOGDailyClose, MSFTDailyClose)
print(f"The Pearson correlation coefficient between GOOG and MSFT is: {correlation_value}")

The Pearson correlation coefficient between GOOG and MSFT is//: 0.8976408107388537


In [25]:
# Correlation_Coefficient Function with Lag
# Using the Pearson correlation coefficient to determine correlation between two stocks with a lag

def Correlation_Coefficient_Lag(StockA, StockB, k=0):
  # Calculate the correlation between the 'close' columns with a lag
  StockA_series = StockA['close'].droplevel('ticker')
  StockB_series = StockB['close'].droplevel('ticker')

  # Apply the lag to StockB
  # Shift the StockB series down by k periods.
  # Positive k shifts data points forward, filling early entries with NaN.
  # Negative k shifts data points backward, filling late entries with NaN.
  StockB_lagged = StockB_series.shift(k)

  # Calculate the correlation between the original StockA and the lagged StockB
  # The .corr() method automatically handles NaN values by excluding them
  correlation = StockA_series.corr(StockB_lagged)

  return correlation

# Example usage with a lag of k=1 day
correlation_value_lag1 = Correlation_Coefficient_Lag(GOOGDailyClose, MSFTDailyClose, k=1)
print(f"The Pearson correlation coefficient between GOOG and MSFT with a 1-day lag on MSFT is: {correlation_value_lag1}")

# Example usage with a lag of k=-1 day
correlation_value_lag_neg1 = Correlation_Coefficient_Lag(GOOGDailyClose, MSFTDailyClose, k=-1)
print(f"The Pearson correlation coefficient between GOOG and MSFT with a -1-day lag on MSFT is: {correlation_value_lag_neg1}")

# Example usage with no lag (k=0)
correlation_value_lag0 = Correlation_Coefficient_Lag(GOOGDailyClose, MSFTDailyClose, k=0)
print(f"The Pearson correlation coefficient between GOOG and MSFT with no lag is: {correlation_value_lag0}")

The Pearson correlation coefficient between GOOG and MSFT with a 1-day lag on MSFT is: 0.8962704402662925
The Pearson correlation coefficient between GOOG and MSFT with a -1-day lag on MSFT is: 0.8945621869369759
The Pearson correlation coefficient between GOOG and MSFT with no lag is: 0.8976408107388537


In [30]:
# Get all unique tickers from the DataFrame
all_tickers = nasdaq_stocks_last_5y_aggregated_close.index.get_level_values('ticker').unique()

# Define the range of lag values
lag_range = range(-2, 3) # -5 to 5 inclusive

# Dictionary to store the correlation results
# The structure will be: { (ticker_a, ticker_b, lag_k): { 'correlation': correlation_value, 'n_days': number_of_days_used } }
all_correlations_with_info = {}

# Iterate through all unique tickers as StockA
for ticker_a in all_tickers:
    # Iterate through all unique tickers as StockB
    for ticker_b in all_tickers:
        # Skip if StockA is the same as StockB (correlation with itself is always 1 with lag 0, less meaningful with lag)
        if ticker_a == ticker_b:
            continue

        print(f"Calculating correlations for pair: {ticker_a} (A) and {ticker_b} (B)")

        # Filter the aggregated data for the current ticker pair
        StockA_data = nasdaq_stocks_last_5y_aggregated_close.loc[nasdaq_stocks_last_5y_aggregated_close.index.get_level_values('ticker') == ticker_a]
        StockB_data = nasdaq_stocks_last_5y_aggregated_close.loc[nasdaq_stocks_last_5y_aggregated_close.index.get_level_values('ticker') == ticker_b]

        # Ensure both stocks have data to compare
        if not StockA_data.empty and not StockB_data.empty:

            # Get the 'close' series and drop the ticker level
            StockA_series = StockA_data['close'].droplevel('ticker')
            StockB_series = StockB_data['close'].droplevel('ticker')

            # Iterate through the defined lag range
            for k in lag_range:
                # Apply the lag to StockB
                StockB_lagged = StockB_series.shift(k)

                # Calculate the correlation coefficient
                correlation = StockA_series.corr(StockB_lagged)

                # Determine the number of non-null pairs used for correlation
                # We need to align the indices and then count non-nulls in both series
                aligned_series = pd.concat([StockA_series, StockB_lagged], axis=1).dropna()
                n_days_used = len(aligned_series)

                # Store the result and the number of days used
                all_correlations_with_info[(ticker_a, ticker_b, k)] = {
                    'correlation': correlation,
                    'n_days': n_days_used
                }

        else:
            print(f"Skipping pair {ticker_a} and {ticker_b} due to missing data.")

# You can now access the correlations and the number of days used
# For example, to see correlation between GOOG and MSFT with lag 1 and the number of days used:
# pair_info = all_correlations_with_info.get(('GOOG', 'MSFT', 1))
# if pair_info:
#     print(f"\nCorrelation between GOOG (A) and MSFT (B) with lag 1: {pair_info['correlation']}")
#     print(f"Number of days used in calculation: {pair_info['n_days']}")

# To see correlation between MSFT and GOOG with lag 1 (MSFT is A, GOOG is B, GOOG lagged by 1):
# pair_info = all_correlations_with_info.get(('MSFT', 'GOOG', 1))
# if pair_info:
#     print(f"\nCorrelation between MSFT (A) and GOOG (B) with lag 1: {pair_info['correlation']}")
#     print(f"Number of days used in calculation: {pair_info['n_days']}")

# To iterate through all results for a specific pair (e.g., GOOG and MSFT):
# print("\nCorrelations for GOOG (A) and MSFT (B) across lags:")
# for k in lag_range:
#     pair_info = all_correlations_with_info.get(('GOOG', 'MSFT', k))
#     if pair_info:
#         print(f"  Lag {k}: Correlation = {pair_info['correlation']:.4f}, Days Used = {pair_info['n_days']}")

# print("\nCorrelations for MSFT (A) and GOOG (B) across lags:")
# for k in lag_range:
#     pair_info = all_correlations_with_info.get(('MSFT', 'GOOG', k))
#     if pair_info:
#         print(f"  Lag {k}: Correlation = {pair_info['correlation']:.4f}, Days Used = {pair_info['n_days']}")

Calculating correlations for pair: A (A) and AAL (B)
Calculating correlations for pair: A (A) and AAPL (B)
Calculating correlations for pair: A (A) and ABBV (B)
Calculating correlations for pair: A (A) and ABNB (B)
Calculating correlations for pair: A (A) and ABT (B)
Calculating correlations for pair: A (A) and ACGL (B)
Calculating correlations for pair: A (A) and ACN (B)
Calculating correlations for pair: A (A) and ADBE (B)
Calculating correlations for pair: A (A) and ADI (B)
Calculating correlations for pair: A (A) and ADM (B)
Calculating correlations for pair: A (A) and ADP (B)
Calculating correlations for pair: A (A) and ADSK (B)
Calculating correlations for pair: A (A) and AEE (B)
Calculating correlations for pair: A (A) and AEP (B)
Calculating correlations for pair: A (A) and AES (B)
Calculating correlations for pair: A (A) and AFL (B)
Calculating correlations for pair: A (A) and AIG (B)
Calculating correlations for pair: A (A) and AIZ (B)
Calculating correlations for pair: A (A)

KeyboardInterrupt: 

## Example how to locate in dataframes and in sereies

In [22]:
# GOOGDailyClose_Series = GOOGDailyClose['close']

# x = GOOGDailyClose.loc[('GOOG', 2020, 1, 2), 'close']
# print(x)

# y = GOOGDailyClose_Series.loc[('GOOG', 2020, 1, 2)]
# print(y)

## Example graph

In [23]:
# # from matplotlib import pyplot as plt
# # GOOGDailyClose['close'].plot(kind='line', figsize=(8, 4), title='close')
# # plt.gca().spines[['top', 'right']].set_visible(False)

# from matplotlib import pyplot as plt

# # Create the plot for GOOGDailyClose
# ax = GOOGDailyClose['close'].plot(kind='line', figsize=(8, 4), title='Closing Stock Prices')

# # Add the plot for MSFTDailyClose to the same axes
# MSFTDailyClose['close'].plot(kind='line', ax=ax)

# # Hide the top and right spines
# plt.gca().spines[['top', 'right']].set_visible(False)

# # Add a legend to distinguish the lines
# plt.legend(['GOOG', 'MSFT'])

# Example Call Function from another file
Eample Call Correlation_Coefficient function from another My Drive file

In [24]:
# %run "/content/drive/My Drive/Colab Notebooks/Correlation_Coefficient.ipynb"

# Correlation_Coefficient(GOOGDailyClose, MSFTDaily Close)
